# Team Assignment #2

Team Sakia - Sakshee Patil and Iara Ravagni

# Importing libraries and loading CSV data file

In [97]:
import pandas as pd
import sqlite3

# Step 1: Load the CSV file into a DataFrame
df = pd.read_csv('data/tips.csv')

# Step 2: Connect to SQLite with a timeout
con = sqlite3.connect('tips.db', timeout=10)

# Step 3: Create a table in the SQLite database
df.to_sql('tips', con, if_exists='replace', index=False)

# Step 4: Commit the transaction and close the connection
con.commit()
con.close()

# Display the DataFrame
df

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


# SQL Queries

## Question 1

Retrieve the average tip percentage for each day of the week

In [98]:
con = sqlite3.connect('tips.db')
c = con.cursor()
c.execute('''SELECT day, ROUND(SUM(tip)/SUM(total_bill)*100, 2) AS avg_tip_per_day
          FROM tips
          GROUP BY day''')

df_temp = pd.DataFrame(c.fetchall(), columns=['day', 'average_tip_per_day'])

print(df_temp)

con.close()

    day  average_tip_per_day
0   Fri                15.94
1   Sat                14.64
2   Sun                15.20
3  Thur                15.67


## Question 2

Find the maximum and minimum total bill amounts

In [99]:
con = sqlite3.connect('tips.db')
c = con.cursor()
c.execute('''SELECT MAX(total_bill) AS max, MIN(total_bill) AS min
          FROM tips''')

df_temp = pd.DataFrame(c.fetchall(), columns=['min_total_bill', 'max_total_bill'])

print(df_temp)

con.close()

   min_total_bill  max_total_bill
0           50.81            3.07


## Question 3

Count the number of parties for each size

In [100]:
con = sqlite3.connect('tips.db')
c = con.cursor()
c.execute('''SELECT size, COUNT(size) AS count
          FROM tips
          GROUP BY size''')

df_temp = pd.DataFrame(c.fetchall(), columns=['size', 'count'])

print(df_temp)

con.close()

   size  count
0     1      4
1     2    156
2     3     38
3     4     37
4     5      5
5     6      4


## Question 4

Retrieve the total bill and tip for parties of size 4 or more, where the tip percentage is greater than 15%


In [101]:
con = sqlite3.connect('tips.db')
c = con.cursor()
c.execute('''SELECT size, total_bill, tip, ROUND(tip/total_bill*100, 2) AS tip_percentage
          FROM tips
          WHERE size >= 4 AND tip_percentage > 15''')

df_temp = pd.DataFrame(c.fetchall(), columns=['size','total_bill', 'tip', 'tip_percentage'])

print(df_temp)

con.close()

    size  total_bill   tip  tip_percentage
0      4       25.29  4.71           18.62
1      4       18.43  3.00           16.28
2      4       39.42  7.58           19.23
3      4       30.40  5.60           18.42
4      4       32.40  6.00           18.52
5      4       25.56  4.34           16.98
6      4       18.29  3.76           20.56
7      4       29.93  5.07           16.94
8      6       34.30  6.70           19.53
9      6       27.05  5.00           18.48
10     5       29.85  5.14           17.22
11     4       21.50  3.50           16.28
12     4       23.17  6.50           28.05
13     5       20.69  5.00           24.17
14     4       20.53  4.00           19.48
15     4       25.89  5.16           19.93
16     4       48.33  9.00           18.62


## Question 5

Retrieve the total bill, tip amount, and tip percentage for each combination of day and time, sorted by tip percentage in descending order

In [102]:
con = sqlite3.connect('tips.db')
c = con.cursor()
c.execute('''SELECT day, time, SUM(total_bill), SUM(tip), ROUND(SUM(tip)/SUM(total_bill)*100,2) AS tip_percentage
          FROM tips
          GROUP BY day, time
          ORDER BY tip_percentage DESC''')

df_temp = pd.DataFrame(c.fetchall(), columns=['day','time', 'total_bill', 'total_tip', 'tip_percentage'])

print(df_temp)

con.close()

    day    time  total_bill  total_tip  tip_percentage
0   Fri   Lunch       89.92      16.68           18.55
1  Thur  Dinner       18.78       3.00           15.97
2  Thur   Lunch     1077.55     168.83           15.67
3   Sun  Dinner     1627.16     247.39           15.20
4   Fri  Dinner      235.96      35.28           14.95
5   Sat  Dinner     1778.40     260.40           14.64


## Question 6

Find the average tip percentage for each combination of day, time, and smoker status

In [103]:
con = sqlite3.connect('tips.db')
c = con.cursor()
query = """SELECT day, time, smoker, ROUND(SUM(tip)/SUM(total_bill)*100, 2) as avg_tip_perc
FROM tips
GROUP BY day, time, smoker"""

result = c.execute(query).fetchall()

df_temp = pd.DataFrame(result,
                        columns=["day", "time", 'smoker', 'avg_tip_perc'])
print(df_temp)

con.close()

     day    time smoker  avg_tip_perc
0    Fri  Dinner     No         14.30
1    Fri  Dinner    Yes         15.16
2    Fri   Lunch     No         18.77
3    Fri   Lunch    Yes         18.50
4    Sat  Dinner     No         15.78
5    Sat  Dinner    Yes         13.51
6    Sun  Dinner     No         15.45
7    Sun  Dinner    Yes         14.58
8   Thur  Dinner     No         15.97
9   Thur   Lunch     No         15.62
10  Thur   Lunch    Yes         15.79


## Question 7

Retrieve the total bill, tip amount, and tip percentage for each sex, sorted by total bill in descending order, and limit the results to the top 5 records

In [104]:
con = sqlite3.connect('tips.db')
c = con.cursor()
query = """SELECT sex, total_bill, tip, ROUND((tip/total_bill)*100, 2) as tip_perc
FROM tips
ORDER BY total_bill DESC
LIMIT 5"""

result = c.execute(query).fetchall()
df_temp = pd.DataFrame(result, columns=["sex", "total_bill", 'tip', 'tip_perc'])
print(df_temp)

con.close()

    sex  total_bill    tip  tip_perc
0  Male       50.81  10.00     19.68
1  Male       48.33   9.00     18.62
2  Male       48.27   6.73     13.94
3  Male       48.17   5.00     10.38
4  Male       45.35   3.50      7.72


## Question 8

Find the maximum and minimum tip percentage for each day and time combination, along with the corresponding total bill and tip amount

In [105]:
con = sqlite3.connect('tips.db')
c = con.cursor()

query = """SELECT day, time, 
total_bill, 
tip, 
ROUND(MAX((tip/total_bill)*100), 2) as max_tip, 
ROUND(MIN((tip/total_bill)*100), 2) as min_tip
FROM tips
GROUP BY day, time;"""

result = c.execute(query).fetchall()

df_temp = pd.DataFrame(result, columns=["day", "time", 'total_bill', 'tip', 'max_tip', 'min_tip'])
print(df_temp)

c.close()

    day    time  total_bill   tip  max_tip  min_tip
0   Fri  Dinner       28.97  3.00    26.35    10.36
1   Fri   Lunch       13.42  1.58    25.93    11.77
2   Sat  Dinner       32.83  1.17    32.57     3.56
3   Sun  Dinner       16.99  1.01    71.03     5.94
4  Thur  Dinner       18.78  3.00    15.97    15.97
5  Thur   Lunch       18.64  1.36    26.63     7.30


## Question 9 

Retrieve the total bill, tip amount, and tip percentage for parties of size 4 or more, where the tip percentage is greater than 15%, and the total bill is between $50 and $100

In [106]:
con = sqlite3.connect('tips.db')
c = con.cursor()

query = """SELECT size, total_bill, tip, ROUND((tip/total_bill)*100, 2) as tip_perc
FROM tips
WHERE size >= 4 
AND (tip/total_bill)*100 >15  
AND total_bill > 50 
AND total_bill < 100"""

result = c.execute(query).fetchall()

df_temp = pd.DataFrame(result, columns=["size", "total_bill", 'tip', 'tip_perc'])
print(df_temp)

con.close()

Empty DataFrame
Columns: [size, total_bill, tip, tip_perc]
Index: []


## Question 10

Find the average tip percentage for each combination of day, time, and smoker status, but only include combinations with more than 5 records

In [107]:
con = sqlite3.connect('tips.db')
c = con.cursor()

query = """SELECT day, time, smoker, ROUND((SUM(tip)/SUM(total_bill))*100, 2) as avg_tip_perc
FROM tips
GROUP BY day, time, smoker
HAVING COUNT(*) > 5"""

result = c.execute(query).fetchall()

df_temp = pd.DataFrame(result, columns=["day", "time", 'smoker', 'avg_tip_perc'])
print(df_temp)

con.close()

    day    time smoker  avg_tip_perc
0   Fri  Dinner    Yes         15.16
1   Fri   Lunch    Yes         18.50
2   Sat  Dinner     No         15.78
3   Sat  Dinner    Yes         13.51
4   Sun  Dinner     No         15.45
5   Sun  Dinner    Yes         14.58
6  Thur   Lunch     No         15.62
7  Thur   Lunch    Yes         15.79


## 5 additional queries


Insert an Additional Row/Rows into the Database

In [108]:
con = sqlite3.connect('tips.db')
c = con.cursor()

print("Original number of rows in Dataset: ", len(df))
c.execute("""INSERT INTO tips VALUES (30.1, 4, "female", "No",	"Thu", "Dinner", 2)""")

many_rows = [
    (70.1, 4, 'female', 'No', 'Thu', 'Dinner', 4),
    (50.7, 5.6, 'male', 'No', 'Fri', 'Dinner', 3),
    (35.9, 4.6, 'female', 'Yes', 'Sat', 'Lunch', 2)]

# using field placeholders
result = c.executemany("""INSERT INTO tips VALUES (?, ?, ?, ?, ?, ?, ?)""", many_rows)

df_temp = pd.DataFrame(result)
con.commit()

# Get the updated number of rows
df_temp = pd.read_sql_query("SELECT * FROM tips", con)
print("Modified number of rows in Dataset: ", len(df_temp))

con.close()

Original number of rows in Dataset:  244
Modified number of rows in Dataset:  248


Exploring Subquerying and Nested Queries: Find all sex and smoker values for tips that are greater than the average tip value:

In [109]:
con = sqlite3.connect('tips.db')
c = con.cursor()

query ="""SELECT sex, smoker
FROM tips
WHERE tip > (
SELECT AVG(tip)
FROM tips
)
"""

result = c.execute(query).fetchall()
print(result)

con.close()

[('Male', 'No'), ('Male', 'No'), ('Female', 'No'), ('Male', 'No'), ('Male', 'No'), ('Male', 'No'), ('Female', 'No'), ('Male', 'No'), ('Male', 'No'), ('Female', 'No'), ('Male', 'No'), ('Male', 'No'), ('Male', 'No'), ('Male', 'No'), ('Male', 'No'), ('Male', 'No'), ('Male', 'No'), ('Female', 'No'), ('Male', 'No'), ('Male', 'No'), ('Male', 'No'), ('Male', 'No'), ('Male', 'No'), ('Female', 'No'), ('Male', 'No'), ('Male', 'No'), ('Male', 'No'), ('Male', 'Yes'), ('Male', 'Yes'), ('Male', 'No'), ('Female', 'Yes'), ('Female', 'Yes'), ('Male', 'Yes'), ('Male', 'No'), ('Male', 'No'), ('Male', 'Yes'), ('Female', 'No'), ('Male', 'No'), ('Male', 'No'), ('Male', 'No'), ('Female', 'Yes'), ('Female', 'No'), ('Male', 'Yes'), ('Male', 'Yes'), ('Female', 'Yes'), ('Female', 'No'), ('Male', 'Yes'), ('Male', 'Yes'), ('Male', 'No'), ('Female', 'Yes'), ('Male', 'No'), ('Female', 'No'), ('Female', 'No'), ('Male', 'No'), ('Female', 'No'), ('Female', 'No'), ('Female', 'No'), ('Male', 'No'), ('Male', 'No'), ('Fema

Creating Indexes

In [110]:
con = sqlite3.connect('tips.db')
c = con.cursor()

res = c.execute("""CREATE INDEX IF NOT EXISTS reservation_no ON tips(total_bill)""")
indexes = c.execute("PRAGMA index_list('tips')").fetchall()
con.commit()

print(indexes)

con.close()

[(0, 'reservation_no', 0, 'c', 0)]


Analyzing Query Performance

In [111]:
# SQLite provides the EXPLAIN QUERY PLAN command, which helps analyze the execution plan of a query and identify areas for optimization.
con = sqlite3.connect('tips.db')
c = con.cursor()

query = "SELECT * FROM tips WHERE total_bill > 50"
res = c.execute("EXPLAIN QUERY PLAN " + query)
data = res.fetchall()
print(data)

con.close()

[(3, 0, 0, 'SEARCH tips USING INDEX reservation_no (total_bill>?)')]


Exploring the Self-Join Operation by analyzing tipping patterns between men and women. 

Joins the tips table with itself to compare tips for different sex while matching on other attributes like size, day, smoker, and time.

In [112]:
con = sqlite3.connect('tips.db')
c = con.cursor()

query = """SELECT t1.sex AS sex1, t1.tip AS tip1, t2.sex AS sex2, t2.tip AS tip2
FROM tips t1
INNER JOIN tips t2
ON t1.size = t2.size
AND t1.day = t2.day
AND t1.smoker = t2.smoker
AND t1.time = t2.time
AND t1.sex != t2.sex"""


result = c.execute(query).fetchall()
df_temp = pd.DataFrame(result, columns=["sex 1", "tip 1", 'sex 2', 'tip 2'])
print(df_temp)

con.close()

       sex 1  tip 1   sex 2  tip 2
0     Female   1.01    Male   1.32
1     Female   1.01    Male   1.56
2     Female   1.01    Male   1.57
3     Female   1.01    Male   1.71
4     Female   1.01    Male   1.96
...      ...    ...     ...    ...
1873    Male   1.75  Female   2.47
1874    Male   1.75  Female   2.75
1875    Male   1.75  Female   3.00
1876    Male   1.75  Female   3.00
1877    Male   1.75  Female   4.08

[1878 rows x 4 columns]


## UPDATE

It was determined that there was an error in the database. Please update the record that corresponds to id=10 and set smoker to Yes.

In [113]:
con = sqlite3.connect('tips.db')
c = con.cursor()


c.execute('''SELECT *
          FROM tips
          WHERE ROWID = 10''')

print("Before updating record")
print(c.fetchall())

c.execute('''UPDATE tips
          SET smoker = 'Yes'
          WHERE ROWID = 10''')


c.execute('''SELECT *
          FROM tips
          WHERE ROWID = 10''')

con.commit()

print("After updating record")
print(c.fetchall())

con.close()

Before updating record
[(14.78, 3.23, 'Male', 'No', 'Sun', 'Dinner', 2)]
After updating record
[(14.78, 3.23, 'Male', 'Yes', 'Sun', 'Dinner', 2)]


## DELETE

Delete records from the database that have a total bill that is less than $10.

In [114]:
con = sqlite3.connect('tips.db')
c = con.cursor()

c.execute('''DELETE FROM tips
          WHERE total_bill < 10''')

con.commit()
con.close()